In [92]:
%pip install pandas
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [93]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [94]:
np.set_printoptions(linewidth=np.inf) # Set linewidth to infinity

In [ ]:

# Constants
n = 10  # periods
b = 0.1  # constant in BDT model
face_value = 1
q = 0.5

# Market spot rates (per period compounding)
market_spot_rates = np.array([3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.55, 3.6, 3.65, 3.7])

# Initial guess for ai values (to be optimized)
a_initial = (1 + market_spot_rates)

print("Initial ai values:", a_initial)


Initial ai values: [4.   4.1  4.2  4.3  4.4  4.5  4.55 4.6  4.65 4.7 ]


In [96]:
def build_short_rate_lattice(a, b, n):
    r = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1):
            r[i, j] = a[i] * np.exp(b * (i - 2 * j))
    return r

In [97]:
def build_elementary_price_lattice(r, n):
    Q = np.zeros((n+1, n+1))
    Q[0][0] = 1
    for i in range(1, n+1):
        for j in range(i + 1):
            if j == 0:
                Q[i][j] += 0.5 * Q[i - 1][j] / (1 + r[i - 1][j]/100)
            elif j == i:
                Q[i][j] += 0.5 * Q[i - 1][j - 1] / (1 + r[i - 1][j - 1]/100)
            else:
                Q[i][j] += 0.5 * (Q[i - 1][j - 1] + Q[i - 1][j]) / (1 + r[i - 1][j-1]/100)
    return Q

In [98]:

# --- ZCB PRICES FROM BDT ---
def compute_bdt_ZCB_prices(a):
    r = build_short_rate_lattice(a, b, n)
    Q = build_elementary_price_lattice(r, n)
    Z_bdt = np.array([np.sum(Q[i]) for i in range(1, n+1)])
    return Z_bdt, r, Q

In [99]:

def calibration_objective(a):
    Z_bdt, _, _ = compute_bdt_ZCB_prices(a)
    spot_bdt = np.array([100 * ((1 / Z_bdt[i]) ** (1 / (i + 1)) - 1) for i in range(n)])
    print("BDT Spot Rates:", spot_bdt)
    return np.sum((spot_bdt - market_spot_rates) ** 2)

In [100]:

# Print short rate lattice, Q lattice, and others with initial a
Z_bdt_initial, r_initial, Q_initial = compute_bdt_ZCB_prices(a_initial)

print("Short Rate Lattice (Initial):")
print(r_initial)

print("\nElementary Price Lattice (Q Initial):")
print(Q_initial)

print("\nZCB Prices from BDT (Initial):")
print(Z_bdt_initial)

objective1 = calibration_objective(a_initial)
print("\nObjective Function Value (Initial):", objective1)

Short Rate Lattice (Initial):
[[4.         0.         0.         0.         0.         0.         0.         0.         0.         0.        ]
 [4.3102115  3.90004064 0.         0.         0.         0.         0.         0.         0.         0.        ]
 [4.64171786 4.2        3.80031716 0.         0.         0.         0.         0.         0.         0.        ]
 [4.99588724 4.52046571 4.09028653 3.7010443  0.         0.         0.         0.         0.         0.        ]
 [5.37417214 4.86275204 4.4        3.98128464 3.60241531 0.         0.         0.         0.         0.        ]
 [5.77811438 5.22825409 4.73071993 4.28053241 3.87318589 3.50460352 0.         0.         0.         0.        ]
 [6.14185757 5.55738255 5.02852768 4.55       4.11701025 3.72522493 3.3707229  0.         0.         0.        ]
 [6.52771072 5.90651692 5.34443752 4.83584704 4.37565535 3.95925669 3.5824836  3.24156521 0.         0.        ]
 [6.93698484 6.27684346 5.67952283 5.13904477 4.65       4.2074939

In [101]:
# Run optimization to minimize the squared difference between market and BDT spot rates
result = minimize(
    calibration_objective,
    a_initial,
    method='BFGS',
    options={'disp': True, 'gtol': 1e-10}
)

# Optimized ai values
a_optimized = result.x

a_optimized

BDT Spot Rates: [4.         4.1036311  4.19224016 4.27094154 4.34297047 4.41040011 4.46706102 4.51665341 4.56134057 4.60247046]
BDT Spot Rates: [4.00000001 4.10363111 4.19224016 4.27094155 4.34297047 4.41040012 4.46706102 4.51665341 4.56134057 4.60247046]
BDT Spot Rates: [4.         4.10363111 4.19224016 4.27094155 4.34297047 4.41040012 4.46706102 4.51665341 4.56134057 4.60247046]
BDT Spot Rates: [4.         4.1036311  4.19224016 4.27094155 4.34297047 4.41040012 4.46706102 4.51665341 4.56134057 4.60247046]
BDT Spot Rates: [4.         4.1036311  4.19224016 4.27094155 4.34297047 4.41040012 4.46706102 4.51665341 4.56134057 4.60247046]
BDT Spot Rates: [4.         4.1036311  4.19224016 4.27094154 4.34297047 4.41040012 4.46706102 4.51665341 4.56134057 4.60247046]
BDT Spot Rates: [4.         4.1036311  4.19224016 4.27094154 4.34297047 4.41040012 4.46706102 4.51665341 4.56134057 4.60247046]
BDT Spot Rates: [4.         4.1036311  4.19224016 4.27094154 4.34297047 4.41040011 4.46706102 4.51665341

array([2.99999999, 3.11838332, 3.26805021, 3.43445284, 3.61011625, 3.79105937, 3.64239936, 3.73348524, 3.82577244, 3.9189319 ])

In [102]:
# Recompute all with optimized ai
Z_bdt_opt, r_bdt_opt, Q_bdt_opt = compute_bdt_ZCB_prices(a_optimized)
spot_bdt_opt = np.array([100 * ((1 / Z_bdt_opt[i]) ** (1 / (i + 1)) - 1) for i in range(n)])

print("\nShort Rate Lattice (Optimized):")
print(r_bdt_opt)

print("\nElementary Price Lattice (Q Optimized):")
print(Q_bdt_opt)

print("\nZCB Prices from BDT (Optimized):")
print(Z_bdt_opt)

print("\nBDT Spot Rates (Optimized):")
print(spot_bdt_opt)

squared_diff_opt = (spot_bdt_opt - market_spot_rates) ** 2
print("\nSquared Differences (Optimized):")
print(squared_diff_opt)


Short Rate Lattice (Optimized):
[[2.99999999 0.         0.         0.         0.         0.         0.         0.         0.         0.        ]
 [3.27826625 2.96629797 0.         0.         0.         0.         0.         0.         0.         0.        ]
 [3.61175405 3.26805021 2.95705412 0.         0.         0.         0.         0.         0.         0.        ]
 [3.99026492 3.61054101 3.2669526  2.95606096 0.         0.         0.         0.         0.         0.        ]
 [4.40940594 3.98979549 3.61011625 3.26656826 2.95571319 0.         0.         0.         0.         0.        ]
 [4.86781658 4.40458259 3.98543114 3.60616722 3.26299503 2.95248    0.         0.         0.         0.        ]
 [4.91672486 4.44883662 4.02547384 3.64239936 3.29577923 2.98214437 2.69835581 0.         0.         0.        ]
 [5.29806774 4.79388994 4.33769099 3.92490512 3.55140101 3.21344052 2.90764122 2.63094258 0.         0.        ]
 [5.70738181 5.16425262 4.67280901 4.22813244 3.82577244 3.4617

In [103]:
# Payer Swaption Pricing via Backward Induction — as per correct recursive formula

# Constants
K = 3.9  # Strike rate
notional = 1_000_000
expiry = 3
swap_start = 4
swap_end = 10

# Step 1: Initialize the swap value lattice with zeros
swap_lattice = np.zeros((n, n))

In [104]:

# Step 2: Set terminal values at t = 9 (last short rate time step), which affects cashflow at t=10
for j in range(10):  # at t=9, there are 10 nodes
    r_ij = r_bdt_opt[9, j]
    swap_lattice[9, j] = (r_ij - K) / (1 + r_ij/100)

print("\nSwap Lattice at t=9:")
print(swap_lattice[9])


Swap Lattice at t=9:
[ 2.1160537   1.57371139  1.07777447  0.62470551  0.21116033 -0.1660082  -0.5097499  -0.82281975 -1.10778257 -1.36701932]


In [105]:

# Step 3: Backward induction from t=8 down to t=3 (expiry)
for i in range(8, expiry - 1, -1):  # i = 8,7,...,3
    for j in range(i + 1):
        r_ij = r_bdt_opt[i, j]
        up_val = swap_lattice[i + 1, j + 1]
        down_val = swap_lattice[i + 1, j]
        expected = q * up_val + (1 - q) * down_val
        intrinsic = r_ij - K
        value = (intrinsic + expected) / (1 + r_ij)
        swap_lattice[i, j] = max(value, 0)  # Ensure non-negative value

print("\nSwap Lattice after Backward Induction:")
print(swap_lattice)  # Show only from t=3 onwards


Swap Lattice after Backward Induction:
[[ 0.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.0334463   0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.12223814  0.03104385  0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.19450011  0.10915949  0.02105444  0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.21525973  0.1316891   0.03906869  0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.29856883  0.2152466   0.12218491  0.01954477  0.          0.          0.          0.          

In [106]:
# Step 4: At t = 3 (expiry), calculate expected payoff at each node, multiply by Q(3,j)
swaption_lattice = np.zeros((expiry + 1, expiry + 1))

swaption_lattice[expiry, :expiry + 1] = np.maximum(0, swap_lattice[expiry, :expiry + 1])
        
print("\nSwaption Lattice at t=3:")
print(swaption_lattice)

for i in range (expiry-1, -1, -1):
    for j in range(i+1):
        swaption_lattice[i][j] = 1 / (1 + r_bdt_opt[i, j] / 100) * (q * swaption_lattice[i + 1][j + 1] + (1 - q) * swaption_lattice[i + 1][j])

print("\nSwaption Lattice after Backward Induction:")
print(swaption_lattice)

swaption_value = notional * swaption_lattice[0, 0]  # Value at t=0
print("\nPayer Swaption Price (rounded):", round(swaption_value))
# Final answer
print("Payer Swaption Price (rounded):", round(swaption_value))


Swaption Lattice at t=3:
[[0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]
 [0.0334463 0.        0.        0.       ]]

Swaption Lattice after Backward Induction:
[[0.00379318 0.         0.         0.        ]
 [0.00781394 0.         0.         0.        ]
 [0.01614021 0.         0.         0.        ]
 [0.0334463  0.         0.         0.        ]]

Payer Swaption Price (rounded): 3793
Payer Swaption Price (rounded): 3793
